# AI model for predicting the electricity prices

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

## Loading the data

In [3]:
# path to the CSV file
path = '/Users/johan/Downloads/data.csv'

df = pd.read_csv(path)

## Define the model

In [4]:
# Define the Neural Network
class PricePredictionModel(nn.Module):
    def __init__(self, input_size):
        super(PricePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # First hidden layer
        self.fc2 = nn.Linear(64, 32)  # Second hidden layer
        self.fc3 = nn.Linear(32, 1)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Activation function for hidden layer 1
        x = torch.relu(self.fc2(x))  # Activation function for hidden layer 2
        x = self.fc3(x)  # Output layer (no activation for regression)
        return x

In [ ]:
# Step 1: Prepare the data
# Assuming 'merged_data' is the dataset we already processed
X = df.drop(columns=['SpotPriceEUR', 'SpotPriceDKK', 'HourUTC', 'from'])  # Features (weather data)
y = df['SpotPriceEUR']  # Target (electricity price in EUR)

# Standardize the features for better training performance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Step 2: Initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
model = PricePredictionModel(input_size)

criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 3: Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Step 4: Evaluate the model
model.eval()
with torch.no_grad():
    predicted = model(X_test)
    test_loss = criterion(predicted, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')